In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 275 ms (started: 2021-09-05 13:35:12 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #3 Sept 05, 2021'

#git push origin master
git push

在 PyTorch 中，torch.nn.Module 模型的可学习参数（即权重和偏差）包含在模型的参数中（通过 model.parameters() 访问）。 state_dict 只是一个 Python 字典对象，它将每一层映射到其参数张量。

# 介绍

如果您有兴趣从 PyTorch 保存或加载模型，则 state_dict 是一个完整的实体。 由于 state_dict 对象是 Python 字典，因此它们可以轻松保存、更新、更改和恢复，从而为 PyTorch 模型和优化器添加了大量模块化。 请注意，只有具有可学习参数的层（卷积层、线性层等）和注册缓冲区（batchnorm 的 running_mean）在模型的 state_dict 中有条目。 优化器对象 (torch.optim) 也有一个 state_dict，其中包含有关优化器状态的信息，以及使用的超参数。 在这个秘籍中，我们将看到 state_dict 如何与一个简单的模型一起使用。

# 步骤
* 导入所有必要的库以加载我们的数据
* 定义并初始化神经网络
* 初始化优化器
* 访问模型和优化器 state_dict

1. 导入必要的库来加载我们的数据

对于这个秘籍，我们将使用 torch 及其子公司 torch.nn 和 torch.optim。

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

time: 699 ms (started: 2021-09-05 13:37:51 +08:00)


2. 定义并初始化神经网络

例如，我们将创建一个用于训练图像的神经网络。 要了解更多信息，请参阅定义神经网络食谱。

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
time: 27.5 ms (started: 2021-09-05 13:38:19 +08:00)


3. 初始化优化器

我们将使用动量的 SGD。

In [4]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

time: 1.1 ms (started: 2021-09-05 13:38:40 +08:00)


4. 访问模型和优化器 state_dict

现在我们已经构建了我们的模型和优化器，我们可以理解在它们各自的 state_dict 属性中保留了什么。

In [5]:
# 打印模型 state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

print()

# 打印优化器的 state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])

Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}]
time: 8.86 ms (started: 2021-09-05 13:39:22 +08:00)


此信息与保存和加载模型和优化器以供将来使用相关。

恭喜！ 您已成功在 PyTorch 中使用 state_dict。